<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Daily_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. **Conception de l'Agent de Dispatch d'Urgence**

A. Environnement (Perceptions) :
- Entrées vocales : ton de la voix, stress vocal, clarté de la parole
- Texte transcrit de l'appel
- Données de géolocalisation de l'appelant
- Bruits de fond significatifs
- Données temporelles (heure, date)

B. Outils :
- Système de reconnaissance vocale avancé
- API de triage médical basée sur les symptômes
- Système de géolocalisation
- Base de données des ressources médicales disponibles
- Système de priorisation des urgences
- Protocoles standardisés d'intervention

C. État (State) :
- ID unique de l'appel
- Informations sur l'appelant
- Liste chronologique des symptômes
- Niveau d'urgence évalué
- Décisions prises
- Ressources déjà allouées
- Timeline des actions

D. Prise de Décision :
- Algorithme de triage multicritères
- Système de scoring d'urgence (1-4)
- Arbre décisionnel basé sur les protocoles médicaux
- Système de validation des décisions critiques

In [1]:
from enum import Enum
from dataclasses import dataclass
from typing import List, Dict, Optional
import datetime
import numpy as np

In [2]:
# Définition des énumérations pour les états et niveaux d'urgence
class UrgencyLevel(Enum):
    LOW = 1
    MEDIUM = 2
    HIGH = 3
    CRITICAL = 4

class AgentType(Enum):
    REACTIVE = "reactive"
    DELIBERATIVE = "deliberative"
    HYBRID = "hybrid"

@dataclass
class Location:
    latitude: float
    longitude: float
    address: str

@dataclass
class Caller:
    id: str
    phone: str
    location: Location
    medical_history: Optional[Dict] = None

@dataclass
class EmergencyState:
    call_id: str
    caller: Caller
    symptoms: List[str]
    urgency_level: UrgencyLevel
    decisions: List[str]
    timestamp: datetime.datetime
    resources_allocated: List[str]

In [5]:
class EmergencyDispatchAgent:
    def __init__(self, agent_type: AgentType = AgentType.HYBRID):
        self.agent_type = agent_type
        self.current_state = None
        self.memory = {}  # Pour l'agent hybride/délibératif
        self.symptom_weights = {
            "difficulty_breathing": 0.9,
            "chest_pain": 0.8,
            "unconscious": 1.0,
            "bleeding": 0.7,
            # Ajoutez d'autres symptômes et leurs poids
        }

    def perceive_environment(self, voice_input: str, location_data: Location) -> dict:
        """Analyse l'entrée vocale et les données de localisation"""
        # Simulation de l'analyse vocale et extraction des symptômes
        detected_symptoms = self._analyze_voice_input(voice_input)
        return {
            "symptoms": detected_symptoms,
            "location": location_data,
            "timestamp": datetime.datetime.now()
        }

    def _analyze_voice_input(self, voice_input: str) -> List[str]:
        symptoms_keywords = {
            "unconscious": ["unconscious", "passed out", "not responding"],
            "difficulty_breathing": ["breathing", "shortness of breath"],
            "chest_pain": ["chest pain", "heart", "cardiac"],
            "bleeding": ["bleeding", "blood", "hemorrhage"],
            "fracture": ["broken", "fracture"],
            "fever": ["fever", "temperature"],
            "accident": ["accident", "crash", "collision"]
        }

        detected_symptoms = []
        voice_input = voice_input.lower()

        for symptom, keywords in symptoms_keywords.items():
            if any(keyword in voice_input for keyword in keywords):
                detected_symptoms.append(symptom)

        return detected_symptoms


    def calculate_urgency(self, symptoms: List[str]) -> UrgencyLevel:
        """Calcul plus nuancé du niveau d'urgence"""
        if not symptoms:
            return UrgencyLevel.LOW

        urgency_weights = {
            "unconscious": 1.0,
            "chest_pain": 0.9,
            "difficulty_breathing": 0.8,
            "bleeding": 0.7,
            "accident": 0.7,
            "fracture": 0.5,
            "fever": 0.3
        }

        urgency_score = sum(urgency_weights.get(s, 0) for s in symptoms)

        if urgency_score >= 0.9:
            return UrgencyLevel.CRITICAL
        elif urgency_score >= 0.7:
            return UrgencyLevel.HIGH
        elif urgency_score >= 0.4:
            return UrgencyLevel.MEDIUM
        return UrgencyLevel.LOW


    def update_state(self, perception: dict) -> None:
        """Met à jour l'état de l'agent"""
        urgency = self.calculate_urgency(perception["symptoms"])

        self.current_state = EmergencyState(
            call_id=f"CALL_{datetime.datetime.now().timestamp()}",
            caller=Caller(
                id=f"CALLER_{datetime.datetime.now().timestamp()}",
                phone="",  # À remplir avec les données réelles
                location=perception["location"]
            ),
            symptoms=perception["symptoms"],
            urgency_level=urgency,
            decisions=[],
            timestamp=perception["timestamp"],
            resources_allocated=[]
        )

    def make_decision(self) -> str:
        """Prend une décision basée sur l'état actuel"""
        if self.agent_type == AgentType.REACTIVE:
            return self._reactive_decision()
        elif self.agent_type == AgentType.DELIBERATIVE:
            return self._deliberative_decision()
        else:  # HYBRID
            return self._hybrid_decision()

    def _reactive_decision(self) -> str:
        """Décision immédiate basée uniquement sur l'état actuel"""
        if self.current_state.urgency_level == UrgencyLevel.CRITICAL:
            return "DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY"
        elif self.current_state.urgency_level == UrgencyLevel.HIGH:
            return "DISPATCH_AMBULANCE"
        elif self.current_state.urgency_level == UrgencyLevel.MEDIUM:
            return "SCHEDULE_URGENT_MEDICAL_VISIT"
        return "PROVIDE_FIRST_AID_INSTRUCTIONS"

    def _deliberative_decision(self) -> str:
        """Décision réfléchie basée sur l'analyse complète"""
        symptoms = set(self.current_state.symptoms)

        if "unconscious" in symptoms:
            return "DISPATCH_SPECIALIZED_CARDIAC_UNIT"
        elif "accident" in symptoms:
            return "DISPATCH_TRAUMA_TEAM_AND_AMBULANCE"
        elif "fracture" in symptoms:
            return "COORDINATE_ORTHOPEDIC_EMERGENCY_RESPONSE"
        elif "fever" in symptoms:
            return "SCHEDULE_TELEHEALTH_CONSULTATION"

        return self._reactive_decision()

    def _hybrid_decision(self) -> str:
        """Combine réactivité et réflexion"""
        if self.current_state.urgency_level in [UrgencyLevel.CRITICAL, UrgencyLevel.HIGH]:
            # Réponse rapide pour les urgences
            reactive_response = self._reactive_decision()
            # Ajout d'actions complémentaires
            if "unconscious" in self.current_state.symptoms:
                return f"{reactive_response} + ALERT_CARDIAC_SPECIALIST"
            elif "accident" in self.current_state.symptoms:
                return f"{reactive_response} + NOTIFY_TRAUMA_CENTER"
            return reactive_response
        else:
            # Approche plus réfléchie pour les cas moins urgents
            return self._deliberative_decision()


    def _plan_based_on_history(self, previous_decisions: List[str]) -> str:
        """Planification basée sur l'historique des décisions"""
        # Logique de planification sophistiquée
        return "PLANNED_RESPONSE"

# Exemple d'utilisation
def main():
    # Création de l'agent
    agent = EmergencyDispatchAgent(AgentType.HYBRID)

    # Création de différentes localisations pour les tests
    locations = {
        "paris": Location(
            latitude=48.8566,
            longitude=2.3522,
            address="4B Rue Pierre Guérin, 75016 Paris, France"
        ),
        "lyon": Location(
            latitude=45.7578,
            longitude=4.8320,
            address="69001 Lyon, France"
        ),
        "marseille": Location(
            latitude=43.2965,
            longitude=5.3698,
            address="13001 Marseille, France"
        )
    }

    # Scénarios de test
    test_scenarios = [
        {
            "description": "Cas critique - Arrêt cardiaque",
            "voice_input": "Patient unconscious with chest pain and difficulty breathing",
            "location": locations["paris"]
        },
        {
            "description": "Cas urgent - Accident de la route",
            "voice_input": "Car accident with bleeding but conscious victim",
            "location": locations["lyon"]
        },
        {
            "description": "Cas modéré - Fracture",
            "voice_input": "Broken arm after falling, conscious and stable",
            "location": locations["marseille"]
        },
        {
            "description": "Cas non-urgent - Symptômes grippaux",
            "voice_input": "Fever and cough since yesterday",
            "location": locations["paris"]
        }
    ]

    # Test de chaque scénario
    for scenario in test_scenarios:
        print("\n" + "="*50)
        print(f"SCÉNARIO: {scenario['description']}")
        print("="*50)

        # Perception de l'environnement
        perception = agent.perceive_environment(
            scenario['voice_input'],
            scenario['location']
        )

        # Mise à jour de l'état
        agent.update_state(perception)

        # Affichage des détails
        print(f"Localisation: {scenario['location'].address}")
        print(f"Symptômes détectés: {perception['symptoms']}")
        print(f"Niveau d'urgence: {agent.current_state.urgency_level}")

        # Test avec différents types d'agents
        for agent_type in AgentType:
            agent.agent_type = agent_type
            decision = agent.make_decision()
            print(f"\nDécision ({agent_type.value}): {decision}")

        # Simulation du suivi temporel (pour l'agent hybride)
        agent.agent_type = AgentType.HYBRID
        if agent.current_state.urgency_level in [UrgencyLevel.HIGH, UrgencyLevel.CRITICAL]:
            print("\nSuivi temporel (5 minutes plus tard):")
            # Simulation d'une mise à jour des symptômes
            updated_voice_input = scenario['voice_input'] + " - responding to initial treatment"
            updated_perception = agent.perceive_environment(
                updated_voice_input,
                scenario['location']
            )
            agent.update_state(updated_perception)
            updated_decision = agent.make_decision()
            print(f"Nouvelle décision: {updated_decision}")

if __name__ == "__main__":
    main()




SCÉNARIO: Cas critique - Arrêt cardiaque
Localisation: 4B Rue Pierre Guérin, 75016 Paris, France
Symptômes détectés: ['unconscious', 'difficulty_breathing', 'chest_pain']
Niveau d'urgence: UrgencyLevel.CRITICAL

Décision (reactive): DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY

Décision (deliberative): DISPATCH_SPECIALIZED_CARDIAC_UNIT

Décision (hybrid): DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY + ALERT_CARDIAC_SPECIALIST

Suivi temporel (5 minutes plus tard):
Nouvelle décision: DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY + ALERT_CARDIAC_SPECIALIST

SCÉNARIO: Cas urgent - Accident de la route
Localisation: 69001 Lyon, France
Symptômes détectés: ['bleeding', 'accident']
Niveau d'urgence: UrgencyLevel.CRITICAL

Décision (reactive): DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY

Décision (deliberative): DISPATCH_TRAUMA_TEAM_AND_AMBULANCE

Décision (hybrid): DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY + NOTIFY_TRAUMA_CENTER

Suivi temporel (5 minutes plus tard):
Nouvelle décision: DISPATCH_EMERGENCY_SERVICES

2. **Classification de l'agent**

Je recommande l'agent HYBRIDE pour les raisons suivantes :

- Gestion de la mémoire :
  * Maintient un état actuel (symptômes, localisation, urgence)
  * Conserve l'historique des décisions pour le suivi
  * Exemple : Dans le cas cardiaque, combine la réponse immédiate avec l'alerte spécialisée

- Planification :
  * Adapte la réponse selon le niveau d'urgence
  * Coordonne différents services (ex: DISPATCH_EMERGENCY_SERVICES_IMMEDIATELY + ALERT_CARDIAC_SPECIALIST)
  * Exemple : Pour la fracture, planifie une réponse orthopédique spécifique

- Action en temps réel :
  * Réagit immédiatement aux cas critiques
  * Maintient un suivi temporel (visible dans les suivis à 5 minutes)
  * Exemple : Dans l'accident de la route, dispatch immédiat + notification du centre de traumatologie

3. **Comparaison avec l'agent réactif**

Si nous utilisions un agent purement réactif :

Changements nécessaires :
- Suppression du système de mémoire
- Simplification des décisions (uniquement DISPATCH/SCHEDULE/PROVIDE)
- Élimination du suivi temporel

Trade-offs :
+ Avantages :
  * Réponse plus rapide
  * Moins de ressources computationnelles
  * Architecture plus simple

- Inconvénients :
  * Pas de coordination spécialisée (pas d'ALERT_CARDIAC_SPECIALIST)
  * Pas de suivi de l'évolution
  * Décisions moins adaptées (comme visible dans le cas de la fracture)

4. **Questions de réflexion**

Sans maintien d'état :
- Perte du suivi des cas critiques
- Impossibilité de coordonner les services spécialisés
- Risque de décisions contradictoires
- Exemple : Dans le cas cardiaque, impossibilité de suivre l'évolution du patient

Importance des outils :
1. Sécurité :
   - Standardisation des protocoles d'urgence
   - Réduction des erreurs humaines
   - Traçabilité des décisions

2. Efficacité :
   - Coordination automatique des services
   - Adaptation rapide aux changements de situation
   - Optimisation des ressources

3. Spécialisation :
   - Dispatch ciblé (équipe de traumatologie, spécialiste cardiaque)
   - Réponses adaptées au niveau d'urgence
   - Intégration des protocoles médicaux spécifiques

Les résultats montrent clairement la supériorité de l'agent hybride, particulièrement dans les cas critiques où il combine la rapidité d'action (comme l'agent réactif) avec la spécialisation des réponses (comme l'agent délibératif). Par exemple, dans le cas cardiaque, il fournit à la fois une réponse immédiate et une coordination spécialisée, ce qui optimise les chances de survie du patient.